In [16]:
#%% Imports
import os
import numpy as np
import matplotlib.pyplot as plt
import skimage.io
import spectral
import json

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

import annotation, image_render, hyspec_io, misc

In [3]:
# Paths
json_gray = '/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/M_Annotation/20210825_Olbergholmen_Annotation_Grayscale/label_classes.json'
annotation_dir = '/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/M_Annotation/20210825_Olbergholmen_Annotation_Grayscale/OlbergAreaS'
hyspec_dir = '/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/4_Rad_Georef_SGC'

In [10]:
# Find names of annotated images
ann_im_fullpath = misc.file_pattern_search(annotation_dir,'*.png')
print(ann_im_fullpath)

['/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/M_Annotation/20210825_Olbergholmen_Annotation_Grayscale/OlbergAreaS/OlbergholmenS_Pika_L_1-Radiance From Raw Data-Georectify Airborne Datacube.png', '/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/M_Annotation/20210825_Olbergholmen_Annotation_Grayscale/OlbergAreaS/OlbergholmenS_Pika_L_10-Radiance From Raw Data-Georectify Airborne Datacube.png', '/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/M_Annotation/20210825_Olbergholmen_Annotation_Grayscale/OlbergAreaS/OlbergholmenS_Pika_L_11-Radiance From Raw Data-Georectify Airborne Datacube.png', '/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/M_Annotation/20210825_Olbergholmen_Annotation_Grayscale/OlbergAreaS/OlbergholmenS_Pika_L_12-Radiance From Raw Data-Georectify Airborne Datacube.png', '/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/M_Annotation/20210825_Olbergholmen_

In [14]:
filenames = [os.path.splitext(os.path.basename(filepath))[0] for filepath in ann_im_fullpath]
for filename in filenames:
    print(filename)

OlbergholmenS_Pika_L_1-Radiance From Raw Data-Georectify Airborne Datacube
OlbergholmenS_Pika_L_10-Radiance From Raw Data-Georectify Airborne Datacube
OlbergholmenS_Pika_L_11-Radiance From Raw Data-Georectify Airborne Datacube
OlbergholmenS_Pika_L_12-Radiance From Raw Data-Georectify Airborne Datacube
OlbergholmenS_Pika_L_15-Radiance From Raw Data-Georectify Airborne Datacube
OlbergholmenS_Pika_L_16-Radiance From Raw Data-Georectify Airborne Datacube
OlbergholmenS_Pika_L_17-Radiance From Raw Data-Georectify Airborne Datacube
OlbergholmenS_Pika_L_18-Radiance From Raw Data-Georectify Airborne Datacube
OlbergholmenS_Pika_L_2-Radiance From Raw Data-Georectify Airborne Datacube
OlbergholmenS_Pika_L_23-Radiance From Raw Data-Georectify Airborne Datacube
OlbergholmenS_Pika_L_24-Radiance From Raw Data-Georectify Airborne Datacube
OlbergholmenS_Pika_L_8-Radiance From Raw Data-Georectify Airborne Datacube
OlbergholmenS_Pika_L_9-Radiance From Raw Data-Georectify Airborne Datacube


In [13]:
# Read annotation metadata file, show classes
class_dict = annotation.read_hasty_metadata(json_gray)
for class_name,class_ind in class_dict.items():
    print(class_name + ': ' + str(class_ind))

Sand: 1
Gravel: 2
Mud: 3
Terrestrial: 4
Zostera marina - dominant: 5
Zostera marina - normal: 6
Zostera marina - sparse: 7
Zostera marina with turf algae: 8
Turf algae - dominant: 9
Turf algae - normal: 10
Turf algae - sparse: 11
Rockweed: 12
Rockweed with turf algae: 13
Fucus serratus: 14
Aegagropila (algae ball): 15
Mixed algae classes on soft substrate: 16
Saccharina latissima: 17


In [28]:
test_list = []
test_list.append({'file': filenames[0], 'classes':list(class_dict.keys())})

In [29]:
test_list

[{'file': 'OlbergholmenS_Pika_L_1-Radiance From Raw Data-Georectify Airborne Datacube',
  'classes': ['Sand',
   'Gravel',
   'Mud',
   'Terrestrial',
   'Zostera marina - dominant',
   'Zostera marina - normal',
   'Zostera marina - sparse',
   'Zostera marina with turf algae',
   'Turf algae - dominant',
   'Turf algae - normal',
   'Turf algae - sparse',
   'Rockweed',
   'Rockweed with turf algae',
   'Fucus serratus',
   'Aegagropila (algae ball)',
   'Mixed algae classes on soft substrate',
   'Saccharina latissima']}]

In [32]:
data = []
for file in filenames:
    ann_file = os.path.join(annotation_dir,file + '.png')
    hyspec_file = os.path.join(annotation_dir,file + '.bip.hdr')
    
    # data.append(dictionary with data)

/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/M_Annotation/20210825_Olbergholmen_Annotation_Grayscale/OlbergAreaS/OlbergholmenS_Pika_L_1-Radiance From Raw Data-Georectify Airborne Datacube.png
/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/M_Annotation/20210825_Olbergholmen_Annotation_Grayscale/OlbergAreaS/OlbergholmenS_Pika_L_10-Radiance From Raw Data-Georectify Airborne Datacube.png
/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/M_Annotation/20210825_Olbergholmen_Annotation_Grayscale/OlbergAreaS/OlbergholmenS_Pika_L_11-Radiance From Raw Data-Georectify Airborne Datacube.png
/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/M_Annotation/20210825_Olbergholmen_Annotation_Grayscale/OlbergAreaS/OlbergholmenS_Pika_L_12-Radiance From Raw Data-Georectify Airborne Datacube.png
/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/M_Annotation/20210825_Olbergholmen_Annotation_Gra

In [36]:
for (key,value) in class_dict.items():
    print(key + ': ' + str(value))

Sand: 1
Gravel: 2
Mud: 3
Terrestrial: 4
Zostera marina - dominant: 5
Zostera marina - normal: 6
Zostera marina - sparse: 7
Zostera marina with turf algae: 8
Turf algae - dominant: 9
Turf algae - normal: 10
Turf algae - sparse: 11
Rockweed: 12
Rockweed with turf algae: 13
Fucus serratus: 14
Aegagropila (algae ball): 15
Mixed algae classes on soft substrate: 16
Saccharina latissima: 17
